### LLM Setup

In [2]:
#from langchain_community.llms import Ollama
from langchain_community.embeddings import GPT4AllEmbeddings

gpt4all_embd = GPT4AllEmbeddings(model_name="all-MiniLM-L6-v2.gguf2.f16.gguf")
#llm = Ollama(model="llama2", temperature=0.5)

from langchain_nvidia_ai_endpoints import ChatNVIDIA
from langchain_nvidia_ai_endpoints import NVIDIAEmbeddings

embedder = NVIDIAEmbeddings(model="nvidia/embed-qa-4", nvidia_api_key="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")

llm = ChatNVIDIA(model="meta/llama3-70b-instruct", nvidia_api_key="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", temperature=0.6)

d:\Installs\Miniconda\envs\LLM\Lib\site-packages\langchain_nvidia_ai_endpoints\_common.py:556: UserWarning: Found nvidia/embed-qa-4 in available_models, but type is unknown and inference may fail.
  warnings.warn(


### Utility functions

In [3]:
async def astream(invocable, inputs, *args, **kwargs):
    async for chunk in invocable.astream(inputs, *args, **kwargs):
        print(chunk, end="")

def parse_bullet_points(text):
    bullet_char = '•'
    lines = text.split('\n')

    lines = list(filter(lambda line: line.startswith(bullet_char), lines))
    return text
    return "\n".join(lines)

#### Prompt Chains

In [3]:
from langchain_core.prompts import ChatPromptTemplate

info_prompt_template = ChatPromptTemplate.from_messages([
    ("system", "You are an AI agent tasked with processing chunks from a codebase. You have sound knowledge of C++ and CUDA. Your objective is to thoroughly understand the information contained in each chunk and return a CONCISE summary. You must provide your answer strictly in BULLET POINTS - all output lines must start with the character '•'. Do NOT use any other sentence formatting, and do NOT include any introductory or concluding statements."),
    ("user", "The code chunk you will need to process is:\n{code_chunk}")
])

info_chain = info_prompt_template | llm

In [4]:
from langchain_core.prompts import ChatPromptTemplate

heirarchy_prompt_template = ChatPromptTemplate.from_messages([
    ("system", """You are an AI agent tasked with combining the essence of the below two code inferences into a single, standalone, and comprehensive passage. Combine overlapping details, include all unique information from both summaries, use concise technical language, and ensure consistency in terminology and formatting. Provide your response strictly in BULLET POINTS. All output lines must start with '•'. Do NOT use any other sentence formatting. Do NOT include any introductory or concluding statements..
"""),
    ("user", "The two code inferences you will need to cobine are:\n text 1:{summary_1} \n text 2:{summary_2}")
])

heirarchy_chain = heirarchy_prompt_template | llm

#### Vectorstore database generation

In [4]:
import os, os.path as osp
from langchain_community.vectorstores import Chroma
from langchain_text_splitters import RecursiveCharacterTextSplitter, Language
from langchain_community.document_loaders import TextLoader
from langchain.docstore.document import Document
from tqdm import tqdm
import glob

class KnowledgeBase:
    def __init__(self, repo_path):
        self.db = Chroma(embedding_function=gpt4all_embd, persist_directory=".\\db\\")
        self.language_splitter = RecursiveCharacterTextSplitter.from_language(language=Language.CPP, chunk_size=1000, chunk_overlap=100)
        self.repo_path = repo_path
        self.in_memory_db = []
        self.per_file_interpretations = []

    def add_knowledge_per_file(self, file_path):
        print(f"Processing {file_path}...")
        interpretations_per_chunk = []
        loader = TextLoader(file_path)
        file_data = loader.load()
        code_chunks = self.language_splitter.split_documents(file_data)

        for chunk in tqdm(code_chunks):
            interpretation = info_chain.invoke(chunk.page_content)
            interpretation_pointwise = parse_bullet_points(interpretation.content)
            print("****************************")
            print(interpretation_pointwise)
            print("****************************")
            source_file_name = chunk.metadata["source"]
            interpretations_per_chunk.append(Document(page_content=source_file_name+"\n"+interpretation_pointwise, metadata=chunk.metadata))
        
        read_buffer = interpretations_per_chunk.copy()
        merge_buffer = []

        while len(read_buffer) > 1:
            for i in range(0, len(read_buffer), 2):
                if i == len(read_buffer)-1:
                    merge_buffer.append(Document(page_content=read_buffer[i].page_content, metadata=read_buffer[i].metadata))
                else:
                    interpretation = heirarchy_chain.invoke({"summary_1":read_buffer[i].page_content, "summary_2":read_buffer[i+1].page_content})
                    interpretation_pointwise = parse_bullet_points(interpretation.content)
                    source_file = read_buffer[i].metadata["source"]
                    print("****************************")
                    print(interpretation_pointwise)
                    print("****************************")
                    merge_buffer.append(Document(page_content=source_file+"\n"+interpretation_pointwise, metadata=read_buffer[i].metadata))

            read_buffer.clear()
            read_buffer.extend(merge_buffer)
            self.in_memory_db.extend(merge_buffer)
            merge_buffer.clear()

        self.per_file_interpretations.append(read_buffer[0])

    def generate_knowledge_base(self):
        
        source_files = glob.glob(self.repo_path + "\\*.*")

        for source_file in source_files:
            self.add_knowledge_per_file(source_file)
        
        read_buffer = self.per_file_interpretations.copy()
        merge_buffer = []

        while len(read_buffer) > 1:
            for i in range(0, len(read_buffer), 2):
                if i == len(read_buffer)-1:
                    merge_buffer.append(Document(page_content=read_buffer[i].page_content, metadata=read_buffer[i].metadata))
                else:
                    interpretation = heirarchy_chain.invoke({"summary_1":read_buffer[i].page_content, "summary_2":read_buffer[i+1].page_content})
                    interpretation_pointwise = parse_bullet_points(interpretation.content)
                    source_list_set = set(read_buffer[i].metadata["source"].split("|")).union(read_buffer[i+1].metadata["source"].split("|"))
                    source_list_str = "|".join(source_list_set)
                    combined_meta = {"source": source_list_str}

                    merge_buffer.append(Document(page_content=source_list_str+"\n"+interpretation_pointwise, metadata=combined_meta))

            read_buffer.clear()
            read_buffer.extend(merge_buffer)
            self.in_memory_db.extend(merge_buffer)
            merge_buffer.clear()

        import pickle

        with open(".\\temp\\in_memory_db_new.pkl", "wb") as pkl_file:
            pickle.dump(self.in_memory_db, pkl_file)
        self.db.add_documents(self.in_memory_db)

In [6]:
repo_path = ".\\PathTracerAP"

knowledge_base = KnowledgeBase(repo_path)
knowledge_base.generate_knowledge_base()

Processing .\PathTracerAP\Config.h...


100%|██████████| 1/1 [00:02<00:00,  3.00s/it]


****************************
• The code defines several constants and macros for a C++ program.
• EPSILON is set to 0.005f, FLOAT_MAX to 9999999.0f, and FLOAT_MIN to -9999990.0f.
• GRID dimensions are set to 25x25x25.
• RESOLUTION is set to 1000x800, with SAMPLES set to 1x1.
• BASE_MODEL_SCALE is set to 1000.
• ITER (iteration) is set to 1000.
• If ENABLE_VISUALIZER is defined, VISUALIZER_WIN dimensions are set to 1600x1200 and MODEL_TOP_VIEW is enabled.
****************************
Processing .\PathTracerAP\Debug_Visualizer.h...


100%|██████████| 1/1 [00:02<00:00,  2.34s/it]


****************************
• This is a header file, indicated by the #pragma once directive.
• It includes several library headers: GLEW, GLFW, and GLM (with additional GLM extensions).
• The included libraries suggest that the code is related to graphics rendering and possibly OpenGL.
• The file includes a custom "Renderer.h" header, implying a dependency on a Renderer class or functionality.
• The code contains a conditional compilation block (#ifdef) that checks for the ENABLE_VISUALIZER flag.
• If ENABLE_VISUALIZER is defined, the code declares a function launch_visualizer that takes a RenderData pointer as an argument.
****************************
Processing .\PathTracerAP\GPUKernels.cuh...


...

****************************
• The function `calculateCoatScattering` takes three inputs: incident direction, surface normal, and refractive index.
• It calculates the reflectance using the `fresnel_reflectance` function.
• It generates a random float between 0 and 1 using the `thrust::uniform_real_distribution` class.
• If the random float is less than the reflectance, it reflects the ray using the `reflectRay` function and sets the mask to white.
• Otherwise, it calculates a random direction in the hemisphere using the `calculateRandomDirectionInHemisphere` function.
• The function `calculateMetalScattering` takes four inputs: normal, ray direction, Phong exponent, and a random number generator.
• It generates a random float between 0 and 1 using the `thrust::uniform_real_distribution` class.
****************************


100%|██████████| 8/8 [00:24<00:00,  3.02s/it]

****************************
• The code calculates a direction vector using importance sampling.
• It first generates a random variable `phi` between 0 and 2*PI.
• It then generates another random variable `importance_sampled_cosine` between 0 and 1.
• The `cosTheta` value is calculated using the `importance_sampled_cosine` and `phong_exponent`.
• The `sinTheta` value is calculated using the `cosTheta` value.
• It defines an orthonormal basis `u`, `v`, `w` using the reflection of the input ray direction and the normal.
• The final direction vector is calculated using the `u`, `v`, `w` basis and the `phi`, `cosTheta`, `sinTheta` values.
****************************


****************************
• This code chunk is a header file, indicated by the #pragma once directive.
• It includes various headers for CUDA, GLM, Thrust, and iostream.
• It defines several macros for mathematical operations, such as MAX, MIN, ABS, IS_EQUAL, IS_LESS_THAN, IS_MORE_THAN, CLAMP, and CEIL.
• It defines constants for PI, TWO_PI, and SQRT_OF_ONE_THIRD.
• The GLM_FORCE_CUDA macro is defined, which suggests that the GLM library is being forced to use CUDA.
• The EPSILON value is used in several macros, but its value is not defined in this chunk.
• The code defines a function `printCUDAMemoryInfo()` to display the free and total GPU memory.
• It uses `cudaMemGetInfo()` to retrieve the free and total bytes of GPU memory.
• If `cudaMemGetInfo()` fails, it prints an error message to `std::cerr`.
• If successful, it prints the free and total GPU memory in megabytes to `std::cout`.
• The code also defines an inline function `utilHash(unsigned int a)` to compute a hash value.
• T

Exception: [404] Not Found
Inference error
RequestID: 806f27ae-4052-4730-9d2a-0ae311df7d80

In [8]:
store_db = Chroma.from_documents(knowledge_base.in_memory_db, embedding=gpt4all_embd, persist_directory=".\\store_db\\vector_store")

In [5]:
store_db_1 = Chroma(embedding_function=gpt4all_embd, persist_directory=".\\store_db\\vector_store")

In [10]:
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_core.runnables import RunnablePassthrough

qa_chat_prompt_template = ChatPromptTemplate.from_messages([
    ("system", "You are an AI assistant that has received a query from a user related to a code base that you have understood thoroughly.You are also well-versed in c++ and CUDA. The query might involve details such as what a specific function does, the purpose of a line of code, how a component fits into the architecture, or any other technical aspect. You will be given a supporting context to answer the query."),
    ("human", "Answer the query below using the provided context:\nQuery:{query}\nContext:{context}")
])

threshold_score = 0.2
info_retriever = store_db_1.as_retriever(
    search_type="similarity_score_threshold",
    search_kwargs={'k': 4, 'score_threshold': threshold_score}
)
qa_chain = ({"query": RunnablePassthrough(), "context": info_retriever } | qa_chat_prompt_template | llm)
qa_chain.invoke("What is the purpose of stencil kernel in renderLoop?").content

'The purpose of the stencil kernel in the `renderLoop` is to compact a stencil buffer based on the remaining bounces of rays. Specifically, it sets stencil values to 0 if the remaining bounces are 0, and 1 otherwise. This kernel is executed on the GPU and is called from the host code. It is an essential part of the rendering process in the PathTracerAP project.'

In [12]:
store_db.similarity_search_with_relevance_scores("What is the use of dev_stencil?")

[(Document(page_content='.\\PathTracerAP\\Renderer.cpp\n• The code chunks from both summaries process different functionalities related to the PathTracerAP renderer.\n• The first summary deals with processing a compact stencil kernel, while the second summary handles a single iteration of a loop.\n• The parameters for the stencil kernel include `compactStencilKernel`, `blocks`, `threads`, `nrays`, and `render_data.dev_ray_data->pool` and `render_data.dev_stencil->pool`.\n• The second summary includes the values of `nrays`, `iterationComplete`, `ibounce`, `gatherImageDataKernel`, `cudaDeviceSynchronize()`, `std::chrono::high_resolution_clock`, `std::chrono::duration_cast`, `intersection time`, and `end time`.\n• The first summary includes the creation of a new thread array using `thrust::stable_partition()` with the range of indices `[render_data.dev_ray_data->pool + nrays, render_data.dev_stencil->pool]`.\n• The second summary includes the incrementation of the `ibounce` counter and th

In [13]:
import pickle

with open(".\\temp\\in_memory_db.pk_iter2", "wb") as pkl_file:
    pickle.dump(knowledge_base.in_memory_db, pkl_file)

### CodeDocBot - UI

In [ ]:
import gradio as gr
import time

def process_user_query(message, history):
    if len(history) % 2 == 0:
        return f"Yes, I do think that '{message}'"
    else:
        return "I don't think so"

def process_source_code(dir_path):
    progress = gr.Progress()
    # Simulate file processing with a delay
    for i in range(10):
        time.sleep(0.5)
        progress(i / 10)
        a = True

# Define Gradio interface
with gr.Blocks() as demo:
    gr.Markdown("# CodeDocBot")
    dir_path = gr.Textbox(label="Source-code directory path", placeholder="Enter the path to the folder")
    process_button = gr.Button("Process File")
    output_text = gr.Textbox(label="")
    
    process_button.click(fn=process_source_code, inputs=dir_path, outputs=output_text)

demo.launch()
gr.ChatInterface(process_user_query).launch()
    